# Data preparetion process

> Yes, it could be done in less amount of code, but that way it's much easier to read<br>
> Also it could be done line by line or even in Spark, but our datasets small enough to use them in Pandas

1. Get all dirs for files

In [1]:
import os
os.chdir('../Data/WHO Datasets/Unboxed')
cwd = os.getcwd()

In [ ]:
# We are having datasets with and without metadata. So let's work only with 'DATA' marked datasets
datalist = []
for dirpath, dirname, filenames in os.walk(cwd):
    for file in filenames:
        if file.endswith("Data.csv"):
            datalist.append(os.path.join(dirpath, file))

2. Check what is in these datasets

In [3]:
with open(datalist[1], mode='r') as f:
    print(f.readline())
    print(f.readline())

Series Name,Series Code,Country Name,Country Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],1966 [YR1966],1967 [YR1967],1968 [YR1968],1969 [YR1969],1970 [YR1970],1971 [YR1971],1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],1986 [YR1986],1987 [YR1987],1988 [YR1988],1989 [YR1989],1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021]

"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,Afghanistan,AFG,50.34,50.44

- Because all datasets are from WHO we are can use that as a reference for all of them
- Some datasets have more than 1 series, so for grouping will be much easier to use Pandas

3. Create a list of countries that you're have in any dataset

In [5]:
import pandas as pd

countries = set()

for dataset in datalist:
    df = pd.read_csv(dataset)
    df.drop(df.loc[df['Series Name'] != df['Series Name'].unique()[0]].index, axis=0, inplace=True)
    countries.update(df['Country Code'].tolist())

len(countries)

275

4. Create a list of countries that you're have in any dataset

In [11]:
country_check = {}
for country in countries:
    country_check[country] = 'Not'

for dataset in datalist:
    df = pd.read_csv(dataset)
    df.drop(df.loc[df['Series Name'] != df['Series Name'].unique()[0]].index, axis=0, inplace=True)
    country_list = df['Country Code'].tolist()

    for country in country_list:
        if country_check[country] != 'Delete':
            country_check[country] = 'Have'
    
    for country in country_check:
        if country_check[country] == 'Not':
            country_check[country] = 'Delete'
        elif country_check[country] == 'Have':
            country_check[country] = 'Not'

countries_filtered = set()

for country in country_check:
    if country_check[country] == 'Not':
        countries_filtered.add(country)

len(countries_filtered)

193

5. Get names of countries with void lines and delete them from our countries_filtered set

In [20]:
country_check = {}

for country in countries_filtered:
    country_check[country] = True

for dataset in datalist:
    df = pd.read_csv(dataset, na_values='..')
    df.drop(['Series Name','Series Code','Country Name'], axis=1, inplace=True)
    idx = list(set(df.index) - set(df.drop('Country Code', axis=1).dropna(how='all').index))
    for country in df.iloc[idx]['Country Code'].dropna().to_list():
        country_check[country] = False
    
for country in country_check:
    if country_check[country] == False and country in countries_filtered:
        countries_filtered.remove(country)

len(countries_filtered)

139

6. Choose a period that you're interested in

> Based on the fact that each dataset can cover different periods, let's use smallest timespan in our datasets

In [44]:
period = {}

for dataset in datalist:
    df = pd.read_csv(dataset, na_values='..')
    years = set(df.drop(['Series Name','Series Code','Country Name', 'Country Code'], axis=1).dropna(how='all',axis=1).columns)
    print(df['Series Name'][0],'\n', years)

Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age) 
 {'2005 [YR2005]', '2018 [YR2018]', '2015 [YR2015]', '2000 [YR2000]', '2010 [YR2010]'}
Birth rate, crude (per 1,000 people) 
 {'1965 [YR1965]', '2002 [YR2002]', '1977 [YR1977]', '1978 [YR1978]', '1994 [YR1994]', '2005 [YR2005]', '2006 [YR2006]', '2003 [YR2003]', '2015 [YR2015]', '1991 [YR1991]', '1983 [YR1983]', '1997 [YR1997]', '2009 [YR2009]', '1964 [YR1964]', '1998 [YR1998]', '2010 [YR2010]', '2014 [YR2014]', '2019 [YR2019]', '1975 [YR1975]', '1995 [YR1995]', '1963 [YR1963]', '1987 [YR1987]', '1999 [YR1999]', '1970 [YR1970]', '1960 [YR1960]', '1979 [YR1979]', '1972 [YR1972]', '1980 [YR1980]', '2007 [YR2007]', '2004 [YR2004]', '2013 [YR2013]', '1985 [YR1985]', '1981 [YR1981]', '2018 [YR2018]', '1996 [YR1996]', '1988 [YR1988]', '2020 [YR2020]', '2012 [YR2012]', '1974 [YR1974]', '2011 [YR2011]', '1993 [YR1993]', '2001 [YR2001]', '1967 [YR1967]', '1962 [YR1962]', '2008 [YR2008]', '1969 

As we can see - we will have a very bad time with our datasets in some cases. So we should forget about dataset:<br>
`Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene`

Also some datasets should be filled, like:<br>
`Total alcohol consumption per capita`

We will use period, based on the last dataset. `2000-2018`